<a href="https://colab.research.google.com/github/Shriramprasath/20ITR096/blob/main/review2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin

# Set the search query for apple seeds
search_query = "Single MRI image duchenne muscular dystrophy"

# Set the number of images to download
num_images = 100

# Set the directory to save the downloaded images
save_directory = "/content/drive/MyDrive/Neuro/dhdgen"

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Prepare the search URL
search_url = f"https://www.google.com/search?q={search_query}&tbm=isch"

# Send a GET request to the search URL
response = requests.get(search_url)

# Parse the response content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all image elements in the parsed HTML
image_elements = soup.find_all('img')

# Download the images
downloaded_images = 0
for i, img in enumerate(image_elements):
    # Get the image source URL
    img_url = img['src']

    try:
        # Check if the URL is a relative URL
        if img_url.startswith('/'):
            # Construct the absolute URL using the base URL of the search page
            img_url = urljoin(search_url, img_url)

        # Send a GET request to download the image
        img_response = requests.get(img_url, stream=True)
        img_response.raise_for_status()
        content_type = img_response.headers.get('Content-Type')
        if content_type == 'image/jpeg':
            file_extension = 'jpg'
        elif content_type == 'image/png':
            file_extension = 'png'
        else:
            continue
        # Save the image to the specified directory
        with open(os.path.join(save_directory, f"image{i+1}.jpg"), 'wb') as f:
            for chunk in img_response.iter_content(chunk_size=128):
                f.write(chunk)

        downloaded_images += 1

        if downloaded_images >= num_images:
            break

    except requests.exceptions.HTTPError:
        continue

# Print the number of downloaded images
print("Number of downloaded images:", downloaded_images)

Number of downloaded images: 20


In [ ]:
import os
from sklearn.model_selection import train_test_split

# Define the path to your image dataset directory
dataset_dir = '/content/drive/MyDrive/Neuro/dhdgen'

# List all image file names in the dataset directory
image_files = [file for file in os.listdir(dataset_dir) if file.endswith('.jpg')]  # Adjust the extension if needed

# Split the dataset into training and testing sets
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Define directories for train and test sets
train_dir = '/content/drive/MyDrive/Neuro/dhdgen/test'
test_dir = '/content/drive/MyDrive/Neuro/dhdgen/train'

# Create directories if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move files to train and test directories
for file in train_files:
    src_path = os.path.join(dataset_dir, file)
    dst_path = os.path.join(train_dir, file)
    os.rename(src_path, dst_path)

for file in test_files:
    src_path = os.path.join(dataset_dir, file)
    dst_path = os.path.join(test_dir, file)
    os.rename(src_path, dst_path)

print(f"Number of images in train set: {len(train_files)}")
print(f"Number of images in test set: {len(test_files)}")

Number of images in train set: 16
Number of images in test set: 4


In [ ]:
# rnn




from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, SimpleRNN, Dropout, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RNN
rnn_input = Reshape((1, 128))(x)  # Reshape to (timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic','llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.40 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

# Evaluate the model using true labels and predicted probabilities
y_true = np.array(true_labels)
y_pred = np.array(predicted_probs) <= 0.40
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
87910968/87910968 [==============================] - 1s 0us/step
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 3s 3s/step
Image: image2.jpg - Predicted Accuracy: 0.5236 - Severity: high severe
1/1 [==============================] - 0s 144ms/step
Image: image3.jpg - Predicted Accuracy: 0.4928 - Severity: high severe
1/1 [==============================] - 0s 156ms/step
Image: image17.jpg - Predicted Accuracy: 0.2483 - Severity: less severe
1/1 [==============================] - 0s 157ms/step
Image: image19.jpg - Predicted Accuracy: 0.5955 - Severity: high severe
Processing subsubfolder: dhdleg/test
1/1 [==============================] - 0s 146ms/step
Image: image4.jpg - Predicted Accuracy: 0.4752 - Severity: high severe
1/1 [==============================] - 0s 140ms/step
Image: image5.jpg - Predicted Accuracy: 0.4889 - Severity: 

In [ ]:
# cnn




from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, SimpleRNN, Dropout, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RNN
rnn_input = Reshape((1, 128))(x)  # Reshape to (timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic','llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.60:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.60 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

# Evaluate the model using true labels and predicted probabilities
y_true = np.array(true_labels)
y_pred = np.array(predicted_probs) <= 0.60
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 2s 2s/step
Image: image2.jpg - Predicted Accuracy: 0.3671 - Severity: less severe
1/1 [==============================] - 0s 152ms/step
Image: image3.jpg - Predicted Accuracy: 0.4598 - Severity: less severe
1/1 [==============================] - 0s 167ms/step
Image: image17.jpg - Predicted Accuracy: 0.4090 - Severity: less severe
1/1 [==============================] - 0s 157ms/step
Image: image19.jpg - Predicted Accuracy: 0.3056 - Severity: less severe
Processing subsubfolder: dhdleg/test
1/1 [==============================] - 0s 162ms/step
Image: image4.jpg - Predicted Accuracy: 0.4075 - Severity: less severe
1/1 [==============================] - 0s 312ms/step
Image: image5.jpg - Predicted Accuracy: 0.2439 - Severity: less severe
1/1 [==============================] - 0s 162ms/step


In [ ]:
# rcnn




from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import (
    Dense,
    GlobalAveragePooling2D,
    SimpleRNN,
    Dropout,
    Reshape,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RCNN
rcnn_input = Reshape((1, 128))(x)  # Reshape to (batch_size, timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rcnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic','llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.40 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Percentage: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Percentage: {less_severe_accuracy:.2f}%")

# Evaluate the model using true labels and predicted probabilities
y_true = np.array(true_labels)
y_pred = np.array(predicted_probs) <= 0.40
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 2s 2s/step
Image: image2.jpg - Predicted Accuracy: 0.5692 - Severity: high severe
1/1 [==============================] - 0s 152ms/step
Image: image3.jpg - Predicted Accuracy: 0.3485 - Severity: less severe
1/1 [==============================] - 0s 150ms/step
Image: image17.jpg - Predicted Accuracy: 0.4961 - Severity: high severe
1/1 [==============================] - 0s 140ms/step
Image: image19.jpg - Predicted Accuracy: 0.5117 - Severity: high severe
Processing subsubfolder: dhdleg/test
1/1 [==============================] - 0s 161ms/step
Image: image4.jpg - Predicted Accuracy: 0.4013 - Severity: high severe
1/1 [==============================] - 0s 176ms/step
Image: image5.jpg - Predicted Accuracy: 0.5643 - Severity: high severe
1/1 [==============================] - 0s 154ms/step


In [ ]:
# dnn




from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import (
    Dense,
    GlobalAveragePooling2D,
    Flatten,  # Using Flatten layer instead of Reshape
    Dropout,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

try:
    # Mount Google Drive
    drive.mount('/content/drive')

    # Define main data directory
    neuro_dir = '/content/drive/MyDrive/Neuro'

    # Load pre-trained InceptionV3 model without top layers
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Flatten layer for DNN
    flatten_layer = Flatten()(x)

    # Add Dense layers
    dense_layer1 = Dense(64, activation='relu')(flatten_layer)
    dense_layer2 = Dense(32, activation='relu')(dense_layer1)

    predictions_layer = Dense(1, activation='sigmoid')(dense_layer2)
    model = Model(inputs=base_model.input, outputs=predictions_layer)

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    # Rest of the code remains the same as in the previous example
    # ... (Loop through main subfolders, image processing, exception handling, etc.)

    # Calculate overall accuracies
    total_images = high_severe_count + less_severe_count
    high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
    less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

    print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
    print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

    # Evaluate the model using true labels and predicted probabilities
    y_true = np.array(true_labels)
    y_pred = np.array(predicted_probs) <= 0.40
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))

except Exception as e:
    print(f"An error occurred: {str(e)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Overall High Severe Accuracy: 74.00%
Overall Less Severe Accuracy: 26.00%

Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.26      0.41       100

    accuracy                           0.26       100
   macro avg       0.50      0.13      0.21       100
weighted avg       1.00      0.26      0.41       100



In [ ]:
# knn



from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import (
    Dense,
    GlobalAveragePooling2D,
    SimpleRNN,
    Dropout,
    Reshape,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RCNN
rcnn_input = Reshape((1, 128))(x)  # Reshape to (batch_size, timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rcnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic', 'llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Lists to store features and labels for KNN training
high_severe_features = []
high_severe_labels = []
less_severe_features = []
less_severe_labels = []

# Initialize KNN classifiers
knn_high_severe = KNeighborsClassifier(n_neighbors=3)
knn_less_severe = KNeighborsClassifier(n_neighbors=3)

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
                less_severe_features.append(img_array)
                less_severe_labels.append(true_label)
            else:
                high_severe_count += 1
                high_severe_features.append(img_array)
                high_severe_labels.append(true_label)

            severity = "less severe" if predicted_prob <= 0.60 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Train KNN classifiers if there are enough samples
if len(high_severe_features) >= knn_high_severe.n_neighbors:
    knn_high_severe.fit(np.array(high_severe_features).reshape(-1, 224 * 224 * 3), np.array(high_severe_labels))
if len(less_severe_features) >= knn_less_severe.n_neighbors:
    knn_less_severe.fit(np.array(less_severe_features).reshape(-1, 224 * 224 * 3), np.array(less_severe_labels))

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

# Evaluate the model using true labels and predicted probabilities
y_true = np.array(true_labels)
y_pred = np.array(predicted_probs) <= 0.60
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 2s 2s/step
Image: image2.jpg - Predicted Accuracy: 0.4758 - Severity: less severe
1/1 [==============================] - 0s 154ms/step
Image: image3.jpg - Predicted Accuracy: 0.6673 - Severity: high severe
1/1 [==============================] - 0s 145ms/step
Image: image17.jpg - Predicted Accuracy: 0.5223 - Severity: less severe
1/1 [==============================] - 0s 138ms/step
Image: image19.jpg - Predicted Accuracy: 0.5401 - Severity: less severe
Processing subsubfolder: dhdleg/test
1/1 [==============================] - 0s 144ms/step
Image: image4.jpg - Predicted Accuracy: 0.8164 - Severity: high severe
1/1 [==============================] - 0s 144ms/step
Image: image5.jpg - Predicted Accuracy: 0.5897 - Severity: less severe
1/1 [==============================] - 0s 154ms/step


In [ ]:
#random forest




from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import (
    Dense,
    GlobalAveragePooling2D,
    SimpleRNN,
    Dropout,
    Reshape,
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Load pre-trained InceptionV3 model without top layers
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

# Reshape for RCNN
rcnn_input = Reshape((1, 128))(x)  # Reshape to (batch_size, timesteps, features)

# Add RNN layer
rnn_layer = SimpleRNN(64)(rcnn_input)

predictions_layer = Dense(1, activation='sigmoid')(rnn_layer)
model = Model(inputs=base_model.input, outputs=predictions_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic', 'llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Lists to store features and labels for Random Forest training
high_severe_features = []
high_severe_labels = []
less_severe_features = []
less_severe_labels = []

# Initialize Random Forest classifiers
rf_high_severe = RandomForestClassifier(n_estimators=100)
rf_less_severe = RandomForestClassifier(n_estimators=100)

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values
            img_array = img_array.reshape((1,) + img_array.shape)

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(img_array)
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
                less_severe_features.append(img_array.flatten())
                less_severe_labels.append(true_label)
            else:
                high_severe_count += 1
                high_severe_features.append(img_array.flatten())
                high_severe_labels.append(true_label)

            severity = "less severe" if predicted_prob <= 0.40 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Train Random Forest classifiers if there are enough samples
if len(high_severe_features) >= 100:
    rf_high_severe.fit(np.array(high_severe_features), np.array(high_severe_labels))
if len(less_severe_features) >= 100:
    rf_less_severe.fit(np.array(less_severe_features), np.array(less_severe_labels))

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

# Evaluate the model using true labels and predicted probabilities
y_true = np.array(true_labels)
y_pred = np.array(predicted_probs) <= 0.40
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 3s 3s/step
Image: image2.jpg - Predicted Accuracy: 0.3253 - Severity: less severe
1/1 [==============================] - 0s 149ms/step
Image: image3.jpg - Predicted Accuracy: 0.3088 - Severity: less severe
1/1 [==============================] - 0s 160ms/step
Image: image17.jpg - Predicted Accuracy: 0.5433 - Severity: high severe
1/1 [==============================] - 0s 138ms/step
Image: image19.jpg - Predicted Accuracy: 0.5417 - Severity: high severe
Processing subsubfolder: dhdleg/test
1/1 [==============================] - 0s 149ms/step
Image: image4.jpg - Predicted Accuracy: 0.4844 - Severity: high severe
1/1 [==============================] - 0s 146ms/step
Image: image5.jpg - Predicted Accuracy: 0.4198 - Severity: high severe
1/1 [==============================] - 0s 143ms/step


In [ ]:
# Multi-Layer Perceptron (MLP)






from google.colab import drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
import os
import numpy as np

# Mount Google Drive
drive.mount('/content/drive')

# Define main data directory
neuro_dir = '/content/drive/MyDrive/Neuro'

# Create an MLP model
model = Sequential([
    Flatten(input_shape=(224, 224, 3)),
    Dense(128, activation='relu'),
    Dropout(0.5),  # This line had an indentation issue in your previous message
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# List all main subfolders
main_subfolders = ['dhdleg', 'dhdhead', 'dhdgen', 'dhdpelvic', 'llg']

# Initialize counters for high severe and less severe images
high_severe_count = 0
less_severe_count = 0

# Lists to store true labels and predicted probabilities for evaluation
true_labels = []
predicted_probs = []

# Loop through main subfolders
for main_subfolder in main_subfolders:
    main_data_dir = os.path.join(neuro_dir, main_subfolder)

    subsubfolders = ['train', 'test']

    for subsubfolder in subsubfolders:
        data_dir = os.path.join(main_data_dir, subsubfolder)

        print(f"Processing subsubfolder: {main_subfolder}/{subsubfolder}")

        # List all image filenames in the subfolder
        image_filenames = [filename for filename in os.listdir(data_dir) if filename.lower().endswith(('.jpg', '.jpeg'))]

        if not image_filenames:
            print("No images found in the directory.")
            continue

        for image_filename in image_filenames:
            # Load the image, preprocess it, and make a prediction
            image_path = os.path.join(data_dir, image_filename)
            img = load_img(image_path, target_size=(224, 224))
            img_array = img_to_array(img)
            img_array = img_array / 255.0  # Normalize pixel values

            true_label = 1  # Assuming all are high severe
            true_labels.append(true_label)

            prediction = model.predict(np.expand_dims(img_array, axis=0))
            predicted_prob = prediction[0][0]
            predicted_probs.append(predicted_prob)

            if predicted_prob <= 0.40:
                less_severe_count += 1
            else:
                high_severe_count += 1

            severity = "less severe" if predicted_prob <= 0.40 else "high severe"

            print(f"Image: {image_filename} - Predicted Accuracy: {predicted_prob:.4f} - Severity: {severity}")

# Calculate overall accuracies
total_images = high_severe_count + less_severe_count
high_severe_accuracy = (high_severe_count / total_images) * 100 if total_images > 0 else 0
less_severe_accuracy = (less_severe_count / total_images) * 100 if total_images > 0 else 0

print(f"\nOverall High Severe Accuracy: {high_severe_accuracy:.2f}%")
print(f"Overall Less Severe Accuracy: {less_severe_accuracy:.2f}%")

# Evaluate the model using true labels and predicted probabilities
y_true = np.array(true_labels)
y_pred = np.array(predicted_probs) <= 0.40
print("\nClassification Report:")
print(classification_report(y_true, y_pred, zero_division=0))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing subsubfolder: dhdleg/train
1/1 [==============================] - 0s 69ms/step
Image: image2.jpg - Predicted Accuracy: 0.5054 - Severity: high severe
1/1 [==============================] - 0s 34ms/step
Image: image3.jpg - Predicted Accuracy: 0.4743 - Severity: high severe
1/1 [==============================] - 0s 32ms/step
Image: image19.jpg - Predicted Accuracy: 0.4471 - Severity: high severe
1/1 [==============================] - 0s 28ms/step
Image: image17.jpg - Predicted Accuracy: 0.4724 - Severity: high severe
Processing subsubfolder: dhdleg/test
1/1 [==============================] - 0s 33ms/step
Image: image18.jpg - Predicted Accuracy: 0.6478 - Severity: high severe
1/1 [==============================] - 0s 31ms/step
Image: image4.jpg - Predicted Accuracy: 0.4931 - Severity: high severe
1/1 [==============================] - 0s 32ms/step
Ima

ModuleNotFoundError: ignored